In [2]:
import os
import argparse
import logging
import time
import numpy as np
import numpy.random as npr
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchdiffeq import odeint_adjoint as odeint
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
fps400data = np.load('C:/Users/shiny/Documents/NeuralODE_MutantRodent/All_Rodent_concatenated_csv/JovinData.npy')
fps200data = fps400data[:,::2,:]


datafilepath = 'C:/Users/shiny/Documents/NeuralODE_MutantRodent/All_Rodent_concatenated_csv/All_Rodent_fps200.npy'
np.save(datafilepath, fps200data)

In [ ]:
X = fps200data.copy()
goodpoint = [0, 1, 2, 3, 15, 16, 17, 18, 19, 20]
Multiple_Trials_Concatenated_ExcludingBadIndex = X[:,:,goodpoint]
datafilepath = 'C:/Users/shiny/Documents/NeuralODE_MutantRodent/All_Rodent_concatenated_csv/RodentXcoord_fps200.npy'
np.save(datafilepath, Multiple_Trials_Concatenated_ExcludingBadIndex)

In [2]:
def load_data_normalize(obs_dim, datafilepath):
    data = np.load(datafilepath)
    traj_tot = data.copy()
    traj_tot = traj_tot[:,2000:9000,:]
    data = data[:, 2000:9000, :]
    data = data.reshape(data.shape[0], 7000, obs_dim)
    noise_std = 0.2

    orig_trajs = np.zeros((data.shape[0],data.shape[1],data.shape[2]))
    for i in range(data.shape[0]):
        for j in range(data.shape[2]):
            trajs = data[i,:,j]
            trajs_tot = traj_tot[i,:,j]
            orig_trajs[i,:,j] = (trajs - trajs_tot.mean()) / trajs_tot.std()
            
    #samp_trajs += npr.randn(*samp_trajs.shape) * noise_std #add noise

    return orig_trajs

def takenembedding(data, tau, k, obs_dim, device):
    data_TE = np.zeros((data.shape[0], data.shape[1]-tau*k, data.shape[2]), dtype = object)
    
    for i in range(data.shape[0]):
        for j in range(data.shape[2]):
            for t in range(data.shape[1]-tau*k):
                data_TE[i,t,j] = data[i, t:t+tau*k+1, j][::tau][::-1]
                
    data_TE = np.array(data_TE.tolist())
    data_TE = data_TE.reshape(data_TE.shape[0],data_TE.shape[1],obs_dim*(k+1))
    
    return data_TE
    

In [6]:
npr.seed(42)
tau = 6
k = 9
obs_dim = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

orig_trajs = load_data_normalize(obs_dim, 'C:/Users/shiny/Documents/NeuralODE_MutantRodent/All_Rodent_concatenated_csv/RodentXcoord_fps200.npy')
orig_trajs_TE = takenembedding(orig_trajs, tau, k, obs_dim, device)

NameError: name 'load_data_normalize' is not defined

## Add Speed

In [4]:
path = 'C:/Users/shiny/Documents/NeuralODE_MutantRodent/JovinData_h5'
dir_list = os.listdir(path)

X = np.zeros((orig_trajs_TE.shape[0],orig_trajs_TE.shape[1],2))

for i in range(orig_trajs_TE.shape[0]):
    DLC_index = dir_list[::2][i].find('DLC')
    X[i, :, 0] = dir_list[::2][i][DLC_index-17:DLC_index-5].replace(",", "." ).replace("_", "" )[0:5]
    X[i, :, 1] = dir_list[::2][i][DLC_index-17:DLC_index-5].replace(",", "." ).replace("_", "" )[5:10]
    
orig_trajs_TE = np.concatenate((orig_trajs_TE, X), axis=2)
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device)
orig_trajs_TE = orig_trajs_TE[:, :6800, :]
orig_trajs_TE = orig_trajs_TE.reshape(orig_trajs_TE.shape[0]*34, 200, obs_dim*(k+1)+2)

orig_trajs_TE = orig_trajs_TE.detach().cpu().numpy()
np.save('orig_trajs_TE_tau6k9_Xcoord.npy',orig_trajs_TE)

In [5]:
npr.seed(42)
tau = 6
k = 9
obs_dim = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
orig_trajs_TE = np.load('orig_trajs_TE_tau6k9_Xcoord.npy')
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(401*34*4, 50, obs_dim*(k+1)+2)

perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:15000]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[21000:24000]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_nonoise_Xcoord_tau6k9.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_nonoise_Xcoord_tau6k9.pt')

## Reducing amount to train

In [8]:
orig_trajs_TE = np.load('orig_trajs_TE_tau6k9_Xcoord.npy')
orig_trajs_TE = orig_trajs_TE[:6000]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(6000*4, 50, obs_dim*(k+1)+2)

perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[:15000]
samp_trajs_val_TE = orig_trajs_TE[perm_index[21000:24000]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_nonoise_shortened_Xcoord_tau6k9.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_nonoise_shortened_Xcoord_tau6k9.pt')